# Imports

In [1]:
import importlib
from functools import partial

from torch import nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import copy

from decision_learning.benchmarks.shortest_path_grid.data import genData
from decision_learning.modeling.loss import SPOPlus, get_loss_function
from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.val_metrics import decision_regret
from decision_learning.modeling.train import train, calc_test_regret, init_loss_data_pretraining, filter_kwargs

### Example Solver/Optimization Model
- Below, `shortest_path_solver` is a custom user optimization model specified in the form of a callable function, and its first input argument is the vector of costs. The rest of the input arguments size, sens, need to be pre-set before being passed to `pipeline`, `train`, or any loss function. This can be accomplished using the `partial` python function (see example below). The exact implementation is not important but mainly that it:
    - accepts a costs vector input
    - returns solution (sol) and objective value (obj) for the input cost vector
- Note that `shortest_path_solver` also has two returns: sol, obj

In [2]:
def shortest_path_solver(costs, size, sens = 1e-4):
    # Forward Pass
    starting_ind = 0
    starting_ind_c = 0
    samples = costs.shape[0]
    V_arr = torch.zeros(samples, size ** 2)
    for i in range(0, 2 * (size - 1)):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i + 2, 9 - i - 1)
        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        V_1 = V_arr[:, starting_ind:starting_ind + num_nodes]
        layer_costs = costs[:, starting_ind_c:starting_ind_c + num_arcs]
        l_costs = layer_costs[:, 0::2]
        r_costs = layer_costs[:, 1::2]
        next_V_val_l = torch.ones(samples, num_nodes_next) * float('inf')
        next_V_val_r = torch.ones(samples, num_nodes_next) * float('inf')
        if num_nodes_next > num_nodes:
            next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
            next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
        else:
            next_V_val_l = V_1[:, :num_nodes_next] + l_costs
            next_V_val_r = V_1[:, 1:num_nodes_next + 1] + r_costs
        next_V_val = torch.minimum(next_V_val_l, next_V_val_r)
        V_arr[:, starting_ind + num_nodes:starting_ind + num_nodes + num_nodes_next] = next_V_val

        starting_ind += num_nodes
        starting_ind_c += num_arcs

    # Backward Pass
    starting_ind = size ** 2
    starting_ind_c = costs.shape[1]
    prev_act = torch.ones(samples, 1)
    sol = torch.zeros(costs.shape)
    for i in range(2 * (size - 1), 0, -1):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i, 9 - i + 1)
        V_1 = V_arr[:, starting_ind - num_nodes:starting_ind]
        V_2 = V_arr[:, starting_ind - num_nodes - num_nodes_next:starting_ind - num_nodes]

        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        layer_costs = costs[:, starting_ind_c - num_arcs: starting_ind_c]

        if num_nodes < num_nodes_next:
            l_cs_res = ((V_2[:, :num_nodes_next - 1] - V_1 + layer_costs[:, ::2]) < sens) * prev_act
            r_cs_res = ((V_2[:, 1:num_nodes_next] - V_1 + layer_costs[:, 1::2]) < sens) * prev_act
            prev_act = torch.zeros(V_2.shape)
            prev_act[:, :num_nodes_next - 1] += l_cs_res
            prev_act[:, 1:num_nodes_next] += r_cs_res
        else:
            l_cs_res = ((V_2 - V_1[:, :num_nodes - 1] + layer_costs[:, ::2]) < sens) * prev_act[:, :num_nodes - 1]
            r_cs_res = ((V_2 - V_1[:, 1:num_nodes] + layer_costs[:, 1::2]) < sens) * prev_act[:, 1:num_nodes]
            prev_act = torch.zeros(V_2.shape)
            prev_act += l_cs_res
            prev_act += r_cs_res
        cs = torch.zeros(layer_costs.shape)
        cs[:, ::2] = l_cs_res
        cs[:, 1::2] = r_cs_res
        sol[:, starting_ind_c - num_arcs: starting_ind_c] = cs

        starting_ind = starting_ind - num_nodes
        starting_ind_c = starting_ind_c - num_arcs
    # Dimension (samples, num edges)
    obj = torch.sum(sol * costs, axis=1)
    # Dimension (samples, 1)
    return sol.to(torch.float32), obj.reshape(-1,1).to(torch.float32)

### Create Experiments Grid
This shortest path experiment has two important settings:
- number of samples: less samples means higher error/more noise, more samples means lower error/less noise
- epsilon: noise level on edge costs, can be uniformly distributed multiplicative noise, or normally distributed additive noise
- This example below creates 100 trials for 8 different settings

In [3]:
# control the randomization seeding for pytorch
torch.manual_seed(105)
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

n_arr = [200, 400, 800, 1600] # array of number of samples for an experiment
ep_arr = ['unif', 'normal'] # noise type
trials = 100 # number of trials per setting

# create an array where each item is [number of samples, noise type, trial number] representing an experiment run
exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t]) # add current [number of samples, noise type, trial number] experiment run setting

In [4]:
# setup
sim = 0 # simulation trial number, only show one experiment run for demonstration purposes
exp = exp_arr[sim] # current experiment
num_data = exp[0]  # number of training data
ep_type = exp[1] # noise type of current experiment
trial = exp[2] # trial number of current experiment

# shortest path problem data generation parameters - https://arxiv.org/pdf/2402.03256
grid = (5, 5)  # grid size
num_feat = 5  # size of feature
deg = 6  # polynomial degree in edge cost function
e = .3  # noise width/amount of noise

# path planting for shortest path example - see page 9, subsection "Harder Example with Planted Arcs" in section 4.2 of paper https://arxiv.org/pdf/2402.03256
planted_good_pwl_params = {'slope0':0, # slope of first segment of piecewise linear cost function for "good" edge cost planted
                    'int0':2, # intercept of first segment of piecewise linear cost function for "good" edge cost planted
                    'slope1':0, # slope of second segment of piecewise linear cost function for "good" edge cost planted
                    'int1':2} # intercept of second segment of piecewise linear cost function for "good" edge cost planted
planted_bad_pwl_params = {'slope0':4, # slope of first segment of piecewise linear cost function for "bad" edge cost planted
                    'int0':0, # intercept of first segment of piecewise linear cost function for "bad" edge cost planted
                    'slope1':0, # slope of second segment of piecewise linear cost function for "bad" edge cost planted
                    'int1':2.2} # intercept of second segment of piecewise linear cost function for "bad" edge cost planted
plant_edge = True # to plant edges in shortest path experiment or not

print(f'current experiment setting: number of data points {num_data}, epsilon type {ep_type}, trial number {trial}')

current experiment setting: number of data points 200, epsilon type unif, trial number 0


### Calling `genData` from `decision_learning.benchmarks.shortest_path_grid.data`

In [5]:
# ------------DATA------------
# training data
generated_data = genData(num_data=num_data+200, # number of data points to generate for training set
        num_features=num_feat, # number of features 
        grid=grid, # grid shape
        deg=deg, # polynomial degree
        noise_type=ep_type, # epsilon noise type
        noise_width=e, # amount of noise
        seed=indices_arr[trial], # seed the randomness
        plant_edges=plant_edge, # to plant edges or not
        planted_good_pwl_params=planted_good_pwl_params, # cost function for good edges
        planted_bad_pwl_params=planted_bad_pwl_params) # cost function for bad edges

# testing data
generated_data_test = genData(num_data=10000, # number of data points to generate for test set
        num_features=num_feat, # number of features 
        grid=grid,  # grid shape
        deg=deg,  # polynomial degree
        noise_type=ep_type,  # epsilon noise type
        noise_width=e, # amount of noise
        seed=indices_arr_test[trial],      # seed the randomness
        plant_edges=plant_edge, # to plant edges or not
        planted_good_pwl_params=planted_good_pwl_params, # cost function for good edges
        planted_bad_pwl_params=planted_bad_pwl_params) # cost function for bad edges

### Split Data into train, val and create structured data inputs
For many decision aware loss function experiment processes, we need the following four:
- X: features
- true_cost: true cost associated with each X
- true_sol: true solution of LP given true_cost
- true_obj: true objective of LP given true_cost

In this case, we will need to get the `true_sol` and `true_obj` for each data sample/`true_cost` vector by plugging into our optimization solver. The code block below shows this.

Furthermore, this package's training loop, in order to flexibly handle different loss function signatures and behavior, requires data to be organized as dictionaries mapping key->data where the keys correspond to named arguments in the loss function. This way, the training loop can flexibly inject or remove the correct named arguments to each loss function

In [6]:
# training data - get true_sol, true_obj
sol, obj = shortest_path_solver(costs=generated_data['cost'], size=5) # plug into solver
# get structured data in the form of dictionary
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj}

# ------------------TRAIN/VAL SPLIT--------------------
train_dict = {}
val_dict = {}

# For each (key,value) tuple in final_data, we split into train val split
# using sklearn train_test_split. Because the test_size, random_state seed are
# always the same, ensures each (key,value) are split the same way across indices
# (this behavior has been checked/tested)
for key, value in final_data.items():
    train_data, val_data = train_test_split(value, test_size=200, random_state=42)
    train_dict[key] = train_data
    val_dict[key] = val_data
    
# test data - get true_sol, true_obj and structured data form
sol_test, obj_test = shortest_path_solver(costs=generated_data_test['cost_true'], size=5)
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost_true'],
              'true_sol':sol_test,
              'true_obj':obj_test}

/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_55518/1886646261.py:18: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_55518/1886646261.py:19: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_55518/1886646261.py:21: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_l = V_1[:, :num_nodes_next] + l_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_55518/1886646261.py:22: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in t

## Prediction Model
- Any decision-aware/focused problem will of course need prediction model to predict the cost/coefficient vector given contextual input/features. This example uses a simple `LinearRegression` object implemented within `decision_learning.modeling.models`. 
- The package expects the prediction model to be a PyTorch model since PyTorch offers convenient autograd functionality/allows user to specify custom losses/backwards passes that are found within many decision-aware/focused works.

In [7]:
# ------------prediction model------------
pred_model = LinearRegression(input_dim=generated_data['feat'].shape[1],
                 output_dim=generated_data['cost'].shape[1])

## CosineSurrogate

In [13]:
import importlib

from decision_learning.modeling.train import init_loss_data_pretraining

import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
from decision_learning.modeling.loss import get_loss_function, CosineSurrogateDotProdMSE, CosineSurrogateDotProdVecMag

In [58]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = CosineSurrogateDotProdVecMag(alpha=1)

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [59]:
train_dataset, train_loader = init_loss_data_pretraining(train_data_dict, 
                        dataloader_params={'batch_size':32, 'shuffle':True})  # training data        



In [60]:
true_cost = train_dataset.data['true_cost']
pred = pred_model(train_dataset.data['X'])

In [61]:
loss_fn.reduction = 'none'
loss = loss_fn(pred_cost=pred, true_cost=true_cost)

In [62]:
loss_fn.alpha*torch.sum(pred[0]*pred[0]), torch.sum(pred[0]*true_cost[0]), loss_fn.alpha*torch.sum(pred[0]*pred[0]) - torch.sum(pred[0]*true_cost[0])

(tensor(10.6915, grad_fn=<MulBackward0>),
 tensor(1.4228, grad_fn=<SumBackward0>),
 tensor(9.2686, grad_fn=<SubBackward0>))

# Train Function

In [73]:
import importlib

import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
from decision_learning.modeling.loss import get_loss_function, CosineSurrogateDotProdMSE, CosineSurrogateDotProdVecMag

In [74]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = CosineSurrogateDotProdVecMag(alpha=5)

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [75]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                test_data_dict=final_data_test, # test data dictionary
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=False)

2025-01-22 09:49:27,043 - decision_learning.modeling.train - INFO - Training on device: cpu
Validation Loader: Epoch 1/100: 100%|██████████| 7/7 [00:00<00:00, 2639.59it/s]
2025-01-22 09:49:27,067 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 17.036829803671157, val_metric: -0.33815249707005035, test_regret: 0.22699389178912618
Validation Loader: Epoch 2/100: 100%|██████████| 7/7 [00:00<00:00, 3450.07it/s]
2025-01-22 09:49:27,085 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: -2.4025398322514127, val_metric: -0.3552398346399721, test_regret: 0.18270424998434606
Validation Loader: Epoch 3/100: 100%|██████████| 7/7 [00:00<00:00, 3217.90it/s]
2025-01-22 09:49:27,102 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: -9.259743009294782, val_metric: -0.20400756472815706, test_regret: 0.11131173635513412
Validation Loader: Epoch 4/100: 100%|██████████| 7/7 [00:00<00:00, 3038.41it/s]
2025-01-22 09:49:27,120 - decision_learning.modeling.trai

In [76]:
metrics

,epoch,train_loss,val_metric,test_regret
0,0,17.036830,-0.338152,0.226994
1,1,-2.402540,-0.355240,0.182704
2,2,-9.259743,-0.204008,0.111312
3,3,-11.247399,-0.261308,0.152418
4,4,-13.196653,-0.300556,0.099896
...,...,...,...,...
95,95,-15.103736,-0.224047,0.049576
96,96,-16.278469,-0.225408,0.049576
97,97,-15.504838,-0.254180,0.049576
98,98,-15.517657,-0.236974,0.049576


In [77]:
trained_model

LinearRegression(
  (linear): Linear(in_features=6, out_features=40, bias=True)
)

In [78]:
true_cost = train_dataset.data['true_cost']
pred = pred_model(train_dataset.data['X'])

In [79]:
loss_fn.reduction = 'none'
loss = loss_fn(pred_cost=pred, true_cost=true_cost)

In [80]:
loss_fn.alpha*torch.sum(pred[0:10]*pred[0:10], axis=1), torch.sum(pred[0:10]*true_cost[0:10], axis=1), loss_fn.alpha*torch.sum(pred[0:10]*pred[0:10], axis=1) - torch.sum(pred[0:10]*true_cost[0:10], axis=1)

(tensor([20.2643, 31.2862, 15.0194,  8.9187, 23.7875,  9.6625, 10.1521, 11.1846,
         13.2788, 33.2027], grad_fn=<MulBackward0>),
 tensor([35.8214, 83.6610, 27.1818, 17.7569, 47.2153, 19.6156, 20.3317, 21.8667,
         23.6769, 76.5764], grad_fn=<SumBackward1>),
 tensor([-15.5571, -52.3748, -12.1624,  -8.8382, -23.4278,  -9.9531, -10.1796,
         -10.6821, -10.3982, -43.3737], grad_fn=<SubBackward0>))

In [94]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(pred, true_cost)

In [97]:
output.mean()

tensor(0.9686, grad_fn=<MeanBackward0>)

In [93]:
pred_mag = torch.norm(pred, dim=1)
true_mag = torch.norm(true_cost, dim=1)

torch.sum(pred*true_cost, axis=1)/(pred_mag*true_mag)

tensor([0.9808, 0.9629, 0.9657, 0.9688, 0.9720, 0.9775, 0.9768, 0.9739, 0.9846,
        0.9714, 0.9668, 0.9714, 0.9853, 0.9551, 0.9825, 0.9813, 0.9704, 0.9764,
        0.9806, 0.6674, 0.9758, 0.9731, 0.9824, 0.9794, 0.9495, 0.9517, 0.9810,
        0.9503, 0.9820, 0.9830, 0.9771, 0.9802, 0.9849, 0.9769, 0.9810, 0.9774,
        0.9321, 0.9546, 0.9775, 0.9805, 0.9753, 0.9277, 0.9782, 0.9842, 0.9814,
        0.9762, 0.9556, 0.9763, 0.9768, 0.9855, 0.9786, 0.9784, 0.9823, 0.9563,
        0.9826, 0.9810, 0.9641, 0.9823, 0.9703, 0.9859, 0.9810, 0.9807, 0.9722,
        0.9786, 0.9785, 0.9777, 0.9639, 0.9819, 0.9782, 0.9754, 0.9111, 0.9047,
        0.9006, 0.9767, 0.9805, 0.9737, 0.9795, 0.9848, 0.9840, 0.9834, 0.9761,
        0.9700, 0.9359, 0.9802, 0.9782, 0.9816, 0.9730, 0.9767, 0.9784, 0.9453,
        0.9794, 0.9041, 0.9746, 0.9778, 0.9823, 0.9866, 0.9787, 0.9643, 0.9820,
        0.9708, 0.9737, 0.9601, 0.9001, 0.9823, 0.9614, 0.8899, 0.9755, 0.9807,
        0.9708, 0.9756, 0.9852, 0.9842, 

# Cosine

In [100]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = get_loss_function('Cosine')()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [101]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                test_data_dict=final_data_test, # test data dictionary
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=False)

2025-01-22 09:54:44,766 - decision_learning.modeling.train - INFO - Training on device: cpu
Validation Loader: Epoch 1/100: 100%|██████████| 7/7 [00:00<00:00, 2335.17it/s]
2025-01-22 09:54:44,800 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.5147819327456611, val_metric: -0.3434356341429162, test_regret: 0.1953900160317956
Validation Loader: Epoch 2/100: 100%|██████████| 7/7 [00:00<00:00, 3743.00it/s]
2025-01-22 09:54:44,816 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.05418300947972706, val_metric: -0.2298140437999502, test_regret: 0.145180021205617
Validation Loader: Epoch 3/100: 100%|██████████| 7/7 [00:00<00:00, 3783.52it/s]
2025-01-22 09:54:44,832 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: 0.04024487414530346, val_metric: -0.1952224625098487, test_regret: 0.11774342014530595
Validation Loader: Epoch 4/100: 100%|██████████| 7/7 [00:00<00:00, 4151.60it/s]
2025-01-22 09:54:44,847 - decision_learning.modeling.train - 

In [102]:
metrics

,epoch,train_loss,val_metric,test_regret
0,0,0.514782,-0.343436,0.195390
1,1,0.054183,-0.229814,0.145180
2,2,0.040245,-0.195222,0.117743
3,3,0.034378,-0.186500,0.092686
4,4,0.029705,-0.218009,0.076176
...,...,...,...,...
95,95,0.022626,-0.197314,0.033418
96,96,0.022198,-0.165810,0.033418
97,97,0.021670,-0.176145,0.033418
98,98,0.021909,-0.200330,0.033418


In [103]:
true_cost = train_dataset.data['true_cost']
pred = trained_model(train_dataset.data['X'])

In [104]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(pred, true_cost)

In [105]:
output.mean()

tensor(0.9778, grad_fn=<MeanBackward0>)